## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-05-03-57-00 +0000,wsj,"UPS Cargo Plane Crash in Kentucky Kills Four, ...",https://www.wsj.com/us-news/ups-cargo-plane-cr...
1,2025-11-05-03-55-45 +0000,bbc,Zohran Mamdani projected winner of New York Ci...,https://www.bbc.com/news/articles/ceq01l8reqlo...
2,2025-11-05-03-55-00 +0000,wsj,Malaysia’s Sunway Healthcare to Begin Pre-Deal...,https://www.wsj.com/finance/malaysias-sunway-h...
3,2025-11-05-03-52-30 +0000,nypost,19-year-old college student from NJ vanishes n...,https://nypost.com/2025/11/04/us-news/19-year-...
4,2025-11-05-03-50-43 +0000,startribune,SNAP benefits delayed with new eligibility rul...,https://www.startribune.com/snap-benefits-minn...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
134,trump,51
12,new,27
16,election,22
505,budget,21
251,cheney,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
371,2025-11-04-10-00-00 +0000,wsj,When the Supreme Court hears arguments over Pr...,https://www.wsj.com/politics/policy/whats-at-s...,141
373,2025-11-04-09-30-53 +0000,bbc,Trump backs Cuomo for New York City mayor and ...,https://www.bbc.com/news/articles/c231e284345o...,140
200,2025-11-04-21-01-07 +0000,nyt,"In New Jersey, Democrats and Republicans say T...",https://www.nytimes.com/live/2025/11/04/us/ele...,129
88,2025-11-05-00-38-00 +0000,wsj,President Trump renominated Jared Isaacman to ...,https://www.wsj.com/politics/elections/trump-r...,123
218,2025-11-04-19-43-00 +0000,wsj,The Trump administration will comply with a co...,https://www.wsj.com/politics/policy/trump-thre...,123


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
371,141,2025-11-04-10-00-00 +0000,wsj,When the Supreme Court hears arguments over Pr...,https://www.wsj.com/politics/policy/whats-at-s...
1,94,2025-11-05-03-55-45 +0000,bbc,Zohran Mamdani projected winner of New York Ci...,https://www.bbc.com/news/articles/ceq01l8reqlo...
260,72,2025-11-04-17-56-00 +0000,wsj,The government pays out $100 billion a year in...,https://www.wsj.com/politics/policy/what-to-kn...
321,64,2025-11-04-12-59-47 +0000,wapo,"Dick Cheney, powerful vice president during wa...",https://www.washingtonpost.com/obituaries/2025...
381,62,2025-11-04-09-00-00 +0000,cbc,Carney's 1st budget calls for billions in new ...,https://www.cbc.ca/news/politics/carney-first-...
18,59,2025-11-05-03-23-05 +0000,startribune,Abigail Spanberger elected Virginia governor i...,https://www.startribune.com/virginia-governors...
8,56,2025-11-05-03-46-30 +0000,nypost,Ex-NJ Gov. James McGreevey headed for runoff i...,https://nypost.com/2025/11/04/us-news/ex-nj-go...
149,50,2025-11-04-22-40-00 +0000,wsj,Major Tesla Investor Rejects Elon Musk’s $1 Tr...,https://www.wsj.com/business/autos/major-tesla...
88,46,2025-11-05-00-38-00 +0000,wsj,President Trump renominated Jared Isaacman to ...,https://www.wsj.com/politics/elections/trump-r...
354,42,2025-11-04-11-00-00 +0000,latimes,California voters head to the polls to decide ...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
